<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/echo/linux-3_10-killer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test is designed to kill Cytoscape v3.10.0-SNAPSHOT on Linux Ubuntu 22.04.1 LTS. It does not kill Cytoscape on a Windows system, and v3.9.1 doesn't die on either Windows or Ubuntu.

The **Network Load test** sends a series of network import and table import messages. This stresses Cytoscape and shows the client's ability to wait a long time for a reply.

I have seen Cytoscape disappear right after using all of available memory. This could be a memory leak issue when importing a .mitab file, possibly an error in the JVM.

I have saved this Notebook with the cell output so you can see what I see. The problem is in the last cell.

If you want to re-execute this workflow, use the Colab "Runtime | Disconnect and delete runtime" before re-excuting. This is a Colab problem. 



---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [1]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cytoscape/py4cytoscape (to revision 1.6.0) to /tmp/pip-req-build-jzrveg9i


  Running command git clone -q https://github.com/cytoscape/py4cytoscape /tmp/pip-req-build-jzrveg9i
  Running command git checkout -b 1.6.0 --track origin/1.6.0
  Switched to a new branch '1.6.0'
  Branch '1.6.0' set up to track remote branch '1.6.0' from 'origin'.


  Created wheel for py4cytoscape: filename=py4cytoscape-1.6.0-py3-none-any.whl size=170883 sha256=4e255e80a70a35469cca45fcc7eedd315cd704353923e5fd2160fa400143e82e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0yei6sdy/wheels/41/f8/55/7d9713a08400a67188976c9214b7e48e0da9cd32b53ad30109
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=cd1b6b1180be4229ec8ff396cdf0ee828200b9b4864aec7eb6796aff26eec9a0
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built py4cytoscape python-igraph
NumExpr defaulting to 2 threads.
Loading Javascript client ... 26f7945d-206c-42c8-a274-06e6fc07e8d8 on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [2]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.10.0-SNAPSHOT',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.6.0',
 'jupyterBridgeVersion': '0.0.2'}

## Setup: Initialize a basic static message
The basic message is a random digit sequence that is unlikely to be used by any other Colab Notebook instance.

In [3]:
import uuid as uu
my_uuid = str(p4c.get_browser_client_channel())

print(f'Fixed message: {my_uuid}')


Fixed message: 26f7945d-206c-42c8-a274-06e6fc07e8d8


## Setup: Prepare Network Load and Column Tests

Load a .mitab network file and .soft attribute file into the Cytoscape sandbox so Cytoscape can read them directly.

Also, define the load_network() function, which is where the trouble actually happens during the test.

In [4]:
res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")

def load_network():
  print(' closing session')
  p4c.close_session(False)
  try:
    print(' importing file')
    p4c.import_network_from_file("BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
  except:
    pass
  print(' importing table')
  p4c.load_table_data_from_file("GDS112_full.soft", start_load_row=83, data_key_column_index=10, delimiters="\t")

## Network Load Test
Load a network and then load the node table data. Both are substantial files that were already staged on the Cytoscape sandbox in a previous step.

This is where death occurs. Most often, Cytoscape just disappears. Sometimes it grabs all memory and then ignores all input. Other times, it can kill the browswer, too.

Given this, I suspect the problem is with the JVM.

In [5]:
%%time

for i in range(0, 10):
  load_network()
  print(f'Load network {i}: {my_uuid} successful')

 closing session
 importing file
 importing table
Load network 0: 26f7945d-206c-42c8-a274-06e6fc07e8d8 successful
 closing session
 importing file
 importing table
Load network 1: 26f7945d-206c-42c8-a274-06e6fc07e8d8 successful
 closing session
 importing file


In commands_post: Could not contact url: http://127.0.0.1:1234/v1/commands/network/load%20file


 importing table


In commands_post: Could not contact url: http://127.0.0.1:1234/v1/commands/table/import%20file


HTTPError: ignored